In [2]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import norm
from sklearn.linear_model import Lasso
from sklearn import datasets

iris = datasets.load_iris()

X = iris.data
y = iris.target

lasso_model = Lasso()

search_space = {'alpha': norm(0, 3)}

metric = 'r2'

search = RandomizedSearchCV(lasso_model,
                            param_distributions=search_space,
                            n_jobs=-1,
                            cv=10,
                            n_iter=50,
                            scoring=metric)

search.fit(X, y)

print(metric)
print(search.best_score_)
print(search.best_params_)

r2
0.15126317207549214
{'alpha': -0.015804785999341483}


/home/nasia/.pyenv/versions/lewagon/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.462e+02, tolerance: 8.333e-03 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent(
/home/nasia/.pyenv/versions/lewagon/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.312e+02, tolerance: 8.333e-03 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one o

In [3]:
import mlflow
from mlflow.tracking import MlflowClient

# Set a remote server to track on
mlflow.set_tracking_uri("https://mlflow.lewagon.co/")

EXPERIMENT_NAME = "[DE] [BERLIN] [atathana] [Ben's file]"

# Create a client
client = MlflowClient()

##################
# Create an experiment
experiment_id = client.create_experiment(
    EXPERIMENT_NAME
)  ## Only on the first run under experiment name, then comment out
#################

##############
# If experiment already exists, retrieve it's id
#current_experiment=dict(mlflow.get_experiment_by_name(EXPERIMENT_NAME))
#experiment_id=current_experiment['experiment_id']
##############

# Run experiment
run = client.create_run(experiment_id)

# Plug in our metric variable and the best score from the random search
client.log_metric(
    run.info.run_id,
    metric,  # our metric
    search.best_score_)  # associated best score

# Plug in the associated parameter values
client.log_param(
    run.info.run_id,
    list(search.best_params_.keys())[0],  # Unpack best parameter key
    list(search.best_params_.values())[0])  # Unpack best parameter value
